# Exercise 2 - Building vulnerbaility functions



In this exercise, we will build a simple vulnerability function using the assumption in impact function from Emanuel (2011), availabile in equation 12 here https://gmd.copernicus.org/articles/12/3085/2019/



### import python libraries

In [ ]:
import math
import pandas as pd

### retrieve intensity bins 

In [ ]:
df_intensity = pd.read_csv('model_data/intensity_bin_dict.csv')

df_intensity

### apply the equation

In [ ]:
# function to calculate damage factor f_ij from max windspeed v_ij
def get_damage_factor(v_ij,v_thresh,v_half):
    v_ij = max(v_ij - v_thresh,0) / (v_half - v_thresh)
    f_ij = (v_ij**3) / (1 + v_ij**3)
    return f_ij

In [ ]:
#vulnerability functions
vulnerability_functions = {'residential': {'vulnerability_id':1, 'v_thresh': 25.7, 'v_half': 74.7}}

vulnerability_functions

In [ ]:
# generate vulnerability file

mps_to_knots = 1.94384

v_lst = []
d_lst = []

for v in vulnerability_functions:
    v_id = vulnerability_functions[v]['vulnerability_id']
    v_thresh = vulnerability_functions[v]['v_thresh'] * mps_to_knots
    v_half = vulnerability_functions[v]['v_half'] * mps_to_knots
    
    for index,row in df_intensity.iterrows():
        intensity = row['intensity_bin_index']
        damage_bin = intensity
        ws_from = (row['max_windspeed'] - 2.5)
        ws_to = (row['max_windspeed'] + 2.5)
        damage_from = get_damage_factor(ws_from,v_thresh,v_half)
        damage_to = get_damage_factor(ws_to,v_thresh,v_half)
        
        v_row = [v_id,intensity,damage_bin]
        v_lst.append(v_row)
        
        d_row = [damage_bin,damage_from,damage_to]
        d_lst.append(d_row)
        
df_vuln = pd.DataFrame(data=v_lst,columns=['vulnerability_id','intensity_bin_index','damage_bin_index'],dtype='int')
df_vuln['prob']=1
df_vuln.to_csv('model_data/vulnerability.csv',index=False)

df_vuln

### discretise the damage factors

In [ ]:
# generate damage file

df_damage = pd.DataFrame(data=d_lst,columns=['damage_bin_index','damage_from','damage_to'])
df_damage['interpolation']=df_damage['damage_from'] + (df_damage['damage_to']-df_damage['damage_from'])/2
df_damage.to_csv('model_data/damage_bin_dict.csv',index=False)

df_damage